# Week 3 - Part 1

## Populate dataframe from wikipedia

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Using BeautifulSoup to download the data

In [188]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tables=soup.find_all('table', class_='wikitable')
table = tables[0]
table = [[t.get_text().strip() for t in tr.find_all('td')] for tr in table.findAll("tr")][1:]
df = pd.DataFrame(table)
df.head()

,0,1,2
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Data Wrangling

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [189]:
df.columns = ['PostalCode', 'Borough','Neighborhood']

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [190]:
df.drop(df[df['Borough'] == 'Not assigned'].index, axis=0, inplace=True)

### More than one neighborhood can exist in one postal code area
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Note: **This Step is not needed anymore, but we implement it anyway**

In [191]:
df.set_index('PostalCode',inplace=True)
df['Neighborhood']=df.groupby('PostalCode')['Neighborhood'].aggregate(', '.join)
df.reset_index(inplace=True)
df.drop_duplicates(subset='PostalCode',inplace=True)

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [192]:
idx=df.Neighborhood == 'Not assigned'
df.loc[idx, 'Neighborhood'] =df.loc[idx, 'Borough']

### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [193]:
df.shape

(103, 3)